line of text used to import the data from the terminal.

mongoimport --type json -d crime -c toronto_crime --drop --jsonArray Neighbourhood_Crime_Rates_Open_Data_chopped.geojson

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'classDB',
 'config',
 'crime',
 'fruitsDB',
 'local',
 'travel_db',
 'uk_food']

In [4]:
# assign the crime database to a variable name
db = mongo['crime']

In [5]:
# review the collections in our new database
db.list_collection_names()

['toronto_crime']

In [6]:
# review a document in the toronto_crime collection
pprint(db.toronto_crime.find_one())

{'_id': ObjectId('65e0a11b8e9820685a9dca82'),
 'geometry': {'coordinates': [[[-79.3974398976879, 43.7069385089713],
                               [-79.3983697420418, 43.7067405816983],
                               [-79.3986310808226, 43.7079940611376],
                               [-79.398681627227, 43.7082183179117],
                               [-79.3988724581394, 43.7090963176949],
                               [-79.3990006746799, 43.7095919421506],
                               [-79.3992914701172, 43.710920903594],
                               [-79.3993199098808, 43.7110852368503],
                               [-79.3994798297642, 43.711847559868],
                               [-79.399566439934, 43.712296943698],
                               [-79.399372772142, 43.7123391241091],
                               [-79.3989526732027, 43.7124163981867],
                               [-79.3985914534404, 43.7124804825853],
                               [-79.3978734488985,

In [7]:
# assign the collection to a variable
toronto_crime = db['toronto_crime']

In [8]:
#selecting the fields for the autotheft data.
query={}
fields={"properties.AREA_NAME":1 , "properties.HOOD_ID":1,"properties.AUTOTHEFT_2014":1,"properties.AUTOTHEFT_2015":1,
       "properties.AUTOTHEFT_2016":1,"properties.AUTOTHEFT_2017":1,"properties.AUTOTHEFT_2018":1,"properties.AUTOTHEFT_2019":1,
       "properties.AUTOTHEFT_2020":1,"properties.AUTOTHEFT_2021":1,"properties.AUTOTHEFT_2022":1,"properties.AUTOTHEFT_2023":1}
auto_theft_data=(list(toronto_crime.find(query,fields)))
auto_theft_data

[{'_id': ObjectId('65e0a11b8e9820685a9dca82'),
  'properties': {'AREA_NAME': 'North Toronto',
   'HOOD_ID': 173,
   'AUTOTHEFT_2014': 4,
   'AUTOTHEFT_2015': 1,
   'AUTOTHEFT_2016': 2,
   'AUTOTHEFT_2017': 2,
   'AUTOTHEFT_2018': 4,
   'AUTOTHEFT_2019': 4,
   'AUTOTHEFT_2020': 11,
   'AUTOTHEFT_2021': 14,
   'AUTOTHEFT_2022': 10,
   'AUTOTHEFT_2023': 18}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca83'),
  'properties': {'AREA_NAME': 'South Eglinton-Davisville',
   'HOOD_ID': 174,
   'AUTOTHEFT_2014': 5,
   'AUTOTHEFT_2015': 4,
   'AUTOTHEFT_2016': 3,
   'AUTOTHEFT_2017': 8,
   'AUTOTHEFT_2018': 15,
   'AUTOTHEFT_2019': 8,
   'AUTOTHEFT_2020': 15,
   'AUTOTHEFT_2021': 15,
   'AUTOTHEFT_2022': 10,
   'AUTOTHEFT_2023': 21}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca84'),
  'properties': {'AREA_NAME': 'Dovercourt Village',
   'HOOD_ID': 172,
   'AUTOTHEFT_2014': 8,
   'AUTOTHEFT_2015': 10,
   'AUTOTHEFT_2016': 18,
   'AUTOTHEFT_2017': 11,
   'AUTOTHEFT_2018': 11,
   'AUTOTHEFT_2019': 12,
  

In [9]:
#converting autotheft data into a pandas dataframe
auto_theft_df=pd.DataFrame(auto_theft_data)
auto_theft_df

,_id,properties
0,65e0a11b8e9820685a9dca82,"{'AREA_NAME': 'North Toronto', 'HOOD_ID': 173,..."
1,65e0a11b8e9820685a9dca83,"{'AREA_NAME': 'South Eglinton-Davisville', 'HO..."
2,65e0a11b8e9820685a9dca84,"{'AREA_NAME': 'Dovercourt Village', 'HOOD_ID':..."
3,65e0a11b8e9820685a9dca85,"{'AREA_NAME': 'Junction-Wallace Emerson', 'HOO..."
4,65e0a11b8e9820685a9dca86,"{'AREA_NAME': 'Yonge-Bay Corridor', 'HOOD_ID':..."
...,...,...
153,65e0a11b8e9820685a9dcb1b,"{'AREA_NAME': 'Humbermede', 'HOOD_ID': 22, 'AU..."
154,65e0a11b8e9820685a9dcb1c,"{'AREA_NAME': 'Kingsview Village-The Westway',..."
155,65e0a11b8e9820685a9dcb1d,"{'AREA_NAME': 'Humber Summit', 'HOOD_ID': 21, ..."
156,65e0a11b8e9820685a9dcb1e,"{'AREA_NAME': 'Pelmo Park-Humberlea', 'HOOD_ID..."


In [10]:
#seperating the info in properties column.(REFERENCE:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)
normalised_df=pd.json_normalize(auto_theft_df['properties'])
auto_theft_df=pd.concat([auto_theft_df['_id'], normalised_df],axis=1)
auto_theft_df

,_id,AREA_NAME,HOOD_ID,AUTOTHEFT_2014,AUTOTHEFT_2015,AUTOTHEFT_2016,AUTOTHEFT_2017,AUTOTHEFT_2018,AUTOTHEFT_2019,AUTOTHEFT_2020,AUTOTHEFT_2021,AUTOTHEFT_2022,AUTOTHEFT_2023
0,65e0a11b8e9820685a9dca82,North Toronto,173,4,1.0,2,2.0,4,4,11,14,10,18
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,5,4.0,3,8.0,15,8,15,15,10,21
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,8,10.0,18,11.0,11,12,18,8,25,28
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,19,17.0,12,17.0,29,23,44,35,47,34
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,12,20.0,20,14.0,27,38,14,17,29,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,36,34.0,47,58.0,57,70,76,61,107,122
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,57,50.0,42,36.0,55,40,60,82,121,112
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,69,41.0,63,83.0,108,134,110,134,174,131
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,29,31.0,26,47.0,67,73,59,67,93,101


In [11]:
#getting an overview of the auto theft data 
auto_theft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             158 non-null    object 
 1   AREA_NAME       158 non-null    object 
 2   HOOD_ID         158 non-null    int64  
 3   AUTOTHEFT_2014  158 non-null    int64  
 4   AUTOTHEFT_2015  157 non-null    float64
 5   AUTOTHEFT_2016  158 non-null    int64  
 6   AUTOTHEFT_2017  157 non-null    float64
 7   AUTOTHEFT_2018  158 non-null    int64  
 8   AUTOTHEFT_2019  158 non-null    int64  
 9   AUTOTHEFT_2020  158 non-null    int64  
 10  AUTOTHEFT_2021  158 non-null    int64  
 11  AUTOTHEFT_2022  158 non-null    int64  
 12  AUTOTHEFT_2023  158 non-null    int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 16.2+ KB


In [12]:
#getting the total value for autotheft
auto_theft_df['Total_AUTOTHEFT']=auto_theft_df[['AUTOTHEFT_2014','AUTOTHEFT_2015','AUTOTHEFT_2016','AUTOTHEFT_2017','AUTOTHEFT_2018',
                 'AUTOTHEFT_2019','AUTOTHEFT_2020','AUTOTHEFT_2021','AUTOTHEFT_2022','AUTOTHEFT_2023']].sum(axis=1)
auto_theft_df

,_id,AREA_NAME,HOOD_ID,AUTOTHEFT_2014,AUTOTHEFT_2015,AUTOTHEFT_2016,AUTOTHEFT_2017,AUTOTHEFT_2018,AUTOTHEFT_2019,AUTOTHEFT_2020,AUTOTHEFT_2021,AUTOTHEFT_2022,AUTOTHEFT_2023,Total_AUTOTHEFT
0,65e0a11b8e9820685a9dca82,North Toronto,173,4,1.0,2,2.0,4,4,11,14,10,18,70.0
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,5,4.0,3,8.0,15,8,15,15,10,21,104.0
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,8,10.0,18,11.0,11,12,18,8,25,28,149.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,19,17.0,12,17.0,29,23,44,35,47,34,277.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,12,20.0,20,14.0,27,38,14,17,29,46,237.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,36,34.0,47,58.0,57,70,76,61,107,122,668.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,57,50.0,42,36.0,55,40,60,82,121,112,655.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,69,41.0,63,83.0,108,134,110,134,174,131,1047.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,29,31.0,26,47.0,67,73,59,67,93,101,593.0


In [13]:
#filling null values with 0
auto_theft_df_clean=auto_theft_df.fillna(value=0)
auto_theft_df_clean

,_id,AREA_NAME,HOOD_ID,AUTOTHEFT_2014,AUTOTHEFT_2015,AUTOTHEFT_2016,AUTOTHEFT_2017,AUTOTHEFT_2018,AUTOTHEFT_2019,AUTOTHEFT_2020,AUTOTHEFT_2021,AUTOTHEFT_2022,AUTOTHEFT_2023,Total_AUTOTHEFT
0,65e0a11b8e9820685a9dca82,North Toronto,173,4,1.0,2,2.0,4,4,11,14,10,18,70.0
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,5,4.0,3,8.0,15,8,15,15,10,21,104.0
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,8,10.0,18,11.0,11,12,18,8,25,28,149.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,19,17.0,12,17.0,29,23,44,35,47,34,277.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,12,20.0,20,14.0,27,38,14,17,29,46,237.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,36,34.0,47,58.0,57,70,76,61,107,122,668.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,57,50.0,42,36.0,55,40,60,82,121,112,655.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,69,41.0,63,83.0,108,134,110,134,174,131,1047.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,29,31.0,26,47.0,67,73,59,67,93,101,593.0


In [14]:
#selecting the fields for the bike theft data.
query={}
fields={"properties.AREA_NAME":1 , "properties.HOOD_ID":1,"properties.BIKETHEFT_2014":1,"properties.BIKETHEFT_2015":1,
       "properties.BIKETHEFT_2016":1,"properties.BIKETHEFT_2017":1,"properties.BIKETHEFT_2018":1,"properties.BIKETHEFT_2019":1,
       "properties.BIKETHEFT_2020":1,"properties.BIKETHEFT_2021":1,"properties.BIKETHEFT_2022":1,"properties.BIKETHEFT_2023":1}
bike_theft_data=(list(toronto_crime.find(query,fields)))
bike_theft_data

[{'_id': ObjectId('65e0a11b8e9820685a9dca82'),
  'properties': {'AREA_NAME': 'North Toronto',
   'HOOD_ID': 173,
   'BIKETHEFT_2014': 5,
   'BIKETHEFT_2015': 13,
   'BIKETHEFT_2016': 9,
   'BIKETHEFT_2017': 13,
   'BIKETHEFT_2018': 17,
   'BIKETHEFT_2019': 22,
   'BIKETHEFT_2020': 50,
   'BIKETHEFT_2021': 22,
   'BIKETHEFT_2022': 10,
   'BIKETHEFT_2023': 22}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca83'),
  'properties': {'AREA_NAME': 'South Eglinton-Davisville',
   'HOOD_ID': 174,
   'BIKETHEFT_2014': 17,
   'BIKETHEFT_2015': 19,
   'BIKETHEFT_2016': 27,
   'BIKETHEFT_2017': 34,
   'BIKETHEFT_2018': 30,
   'BIKETHEFT_2019': 42,
   'BIKETHEFT_2020': 94,
   'BIKETHEFT_2021': 34,
   'BIKETHEFT_2022': 18,
   'BIKETHEFT_2023': 37}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca84'),
  'properties': {'AREA_NAME': 'Dovercourt Village',
   'HOOD_ID': 172,
   'BIKETHEFT_2014': 26,
   'BIKETHEFT_2015': 34,
   'BIKETHEFT_2016': 52,
   'BIKETHEFT_2017': 45,
   'BIKETHEFT_2018': 30,
   'BIKETHEFT_201

In [15]:
#converting biketheft data into a pandas dataframe
bike_theft_df=pd.DataFrame(bike_theft_data)
bike_theft_df

,_id,properties
0,65e0a11b8e9820685a9dca82,"{'AREA_NAME': 'North Toronto', 'HOOD_ID': 173,..."
1,65e0a11b8e9820685a9dca83,"{'AREA_NAME': 'South Eglinton-Davisville', 'HO..."
2,65e0a11b8e9820685a9dca84,"{'AREA_NAME': 'Dovercourt Village', 'HOOD_ID':..."
3,65e0a11b8e9820685a9dca85,"{'AREA_NAME': 'Junction-Wallace Emerson', 'HOO..."
4,65e0a11b8e9820685a9dca86,"{'AREA_NAME': 'Yonge-Bay Corridor', 'HOOD_ID':..."
...,...,...
153,65e0a11b8e9820685a9dcb1b,"{'AREA_NAME': 'Humbermede', 'HOOD_ID': 22, 'BI..."
154,65e0a11b8e9820685a9dcb1c,"{'AREA_NAME': 'Kingsview Village-The Westway',..."
155,65e0a11b8e9820685a9dcb1d,"{'AREA_NAME': 'Humber Summit', 'HOOD_ID': 21, ..."
156,65e0a11b8e9820685a9dcb1e,"{'AREA_NAME': 'Pelmo Park-Humberlea', 'HOOD_ID..."


In [16]:
#seperating the info in properties column.(REFERENCE:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)
normalised_df=pd.json_normalize(bike_theft_df['properties'])
bike_theft_df=pd.concat([bike_theft_df['_id'], normalised_df],axis=1)
bike_theft_df

,_id,AREA_NAME,HOOD_ID,BIKETHEFT_2014,BIKETHEFT_2015,BIKETHEFT_2016,BIKETHEFT_2017,BIKETHEFT_2018,BIKETHEFT_2019,BIKETHEFT_2020,BIKETHEFT_2021,BIKETHEFT_2022,BIKETHEFT_2023
0,65e0a11b8e9820685a9dca82,North Toronto,173,5,13,9,13,17,22,50,22,10,22.0
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,17,19,27,34,30,42,94,34,18,37.0
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,26,34,52,45,30,26,28,26,20,38.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,40,41,50,62,65,70,54,36,36,55.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,223,156,202,215,267,219,173,169,230,228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,2,5,3,3,2,1,4,2,1,2.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,0,6,2,4,5,5,5,3,5,1.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,7,3,6,3,2,7,1,2,2,4.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,3,1,1,0,4,1,5,3,5,NaN


In [17]:
#getting an overview of the bike theft data 
bike_theft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             158 non-null    object 
 1   AREA_NAME       158 non-null    object 
 2   HOOD_ID         158 non-null    int64  
 3   BIKETHEFT_2014  158 non-null    int64  
 4   BIKETHEFT_2015  158 non-null    int64  
 5   BIKETHEFT_2016  158 non-null    int64  
 6   BIKETHEFT_2017  158 non-null    int64  
 7   BIKETHEFT_2018  158 non-null    int64  
 8   BIKETHEFT_2019  158 non-null    int64  
 9   BIKETHEFT_2020  158 non-null    int64  
 10  BIKETHEFT_2021  158 non-null    int64  
 11  BIKETHEFT_2022  158 non-null    int64  
 12  BIKETHEFT_2023  154 non-null    float64
dtypes: float64(1), int64(10), object(2)
memory usage: 16.2+ KB


In [19]:
#getting the total value for biketheft
bike_theft_df['Total_BIKETHEFT']=bike_theft_df[['BIKETHEFT_2014','BIKETHEFT_2015','BIKETHEFT_2016','BIKETHEFT_2017','BIKETHEFT_2018',
                 'BIKETHEFT_2019','BIKETHEFT_2020','BIKETHEFT_2021','BIKETHEFT_2022','BIKETHEFT_2023']].sum(axis=1)
bike_theft_df

,_id,AREA_NAME,HOOD_ID,BIKETHEFT_2014,BIKETHEFT_2015,BIKETHEFT_2016,BIKETHEFT_2017,BIKETHEFT_2018,BIKETHEFT_2019,BIKETHEFT_2020,BIKETHEFT_2021,BIKETHEFT_2022,BIKETHEFT_2023,Total_BIKETHEFT
0,65e0a11b8e9820685a9dca82,North Toronto,173,5,13,9,13,17,22,50,22,10,22.0,183.0
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,17,19,27,34,30,42,94,34,18,37.0,352.0
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,26,34,52,45,30,26,28,26,20,38.0,325.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,40,41,50,62,65,70,54,36,36,55.0,509.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,223,156,202,215,267,219,173,169,230,228.0,2082.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,2,5,3,3,2,1,4,2,1,2.0,25.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,0,6,2,4,5,5,5,3,5,1.0,36.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,7,3,6,3,2,7,1,2,2,4.0,37.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,3,1,1,0,4,1,5,3,5,NaN,23.0


In [20]:
#filling null values with 0
bike_theft_df_clean=bike_theft_df.fillna(value=0)
bike_theft_df_clean

,_id,AREA_NAME,HOOD_ID,BIKETHEFT_2014,BIKETHEFT_2015,BIKETHEFT_2016,BIKETHEFT_2017,BIKETHEFT_2018,BIKETHEFT_2019,BIKETHEFT_2020,BIKETHEFT_2021,BIKETHEFT_2022,BIKETHEFT_2023,Total_BIKETHEFT
0,65e0a11b8e9820685a9dca82,North Toronto,173,5,13,9,13,17,22,50,22,10,22.0,183.0
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,17,19,27,34,30,42,94,34,18,37.0,352.0
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,26,34,52,45,30,26,28,26,20,38.0,325.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,40,41,50,62,65,70,54,36,36,55.0,509.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,223,156,202,215,267,219,173,169,230,228.0,2082.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,2,5,3,3,2,1,4,2,1,2.0,25.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,0,6,2,4,5,5,5,3,5,1.0,36.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,7,3,6,3,2,7,1,2,2,4.0,37.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,3,1,1,0,4,1,5,3,5,0.0,23.0


In [21]:
#selecting fields for shooting data 
query={}
fields={"properties.AREA_NAME":1 , "properties.HOOD_ID":1,"properties.SHOOTING_2014":1,"properties.SHOOTING_2015":1,
       "properties.SHOOTING_2016":1,"properties.SHOOTING_2017":1,"properties.SHOOTING_2018":1,"properties.SHOOTING_2019":1,
       "properties.SHOOTING_2020":1,"properties.SHOOTING_2021":1,"properties.SHOOTING_2022":1,"properties.SHOOTING_2023":1}
shooting_data=(list(toronto_crime.find(query,fields)))
shooting_data

[{'_id': ObjectId('65e0a11b8e9820685a9dca82'),
  'properties': {'AREA_NAME': 'North Toronto',
   'HOOD_ID': 173,
   'SHOOTING_2014': 0,
   'SHOOTING_2015': 0,
   'SHOOTING_2016': 0,
   'SHOOTING_2017': 0,
   'SHOOTING_2018': 0,
   'SHOOTING_2019': 0,
   'SHOOTING_2020': 0,
   'SHOOTING_2021': 1,
   'SHOOTING_2022': 1,
   'SHOOTING_2023': None}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca83'),
  'properties': {'AREA_NAME': 'South Eglinton-Davisville',
   'HOOD_ID': 174,
   'SHOOTING_2014': 1,
   'SHOOTING_2015': 0,
   'SHOOTING_2016': 1,
   'SHOOTING_2017': 0,
   'SHOOTING_2018': 1,
   'SHOOTING_2019': 1,
   'SHOOTING_2020': 1,
   'SHOOTING_2021': 0,
   'SHOOTING_2022': 1,
   'SHOOTING_2023': None}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca84'),
  'properties': {'AREA_NAME': 'Dovercourt Village',
   'HOOD_ID': 172,
   'SHOOTING_2014': 0,
   'SHOOTING_2015': 0,
   'SHOOTING_2016': 2,
   'SHOOTING_2017': 0,
   'SHOOTING_2018': 0,
   'SHOOTING_2019': 0,
   'SHOOTING_2020': 0,
   'SHOOTING_

In [22]:
#converting shooting data into a pandas dataframe
shooting_df=pd.DataFrame(shooting_data)
shooting_df


,_id,properties
0,65e0a11b8e9820685a9dca82,"{'AREA_NAME': 'North Toronto', 'HOOD_ID': 173,..."
1,65e0a11b8e9820685a9dca83,"{'AREA_NAME': 'South Eglinton-Davisville', 'HO..."
2,65e0a11b8e9820685a9dca84,"{'AREA_NAME': 'Dovercourt Village', 'HOOD_ID':..."
3,65e0a11b8e9820685a9dca85,"{'AREA_NAME': 'Junction-Wallace Emerson', 'HOO..."
4,65e0a11b8e9820685a9dca86,"{'AREA_NAME': 'Yonge-Bay Corridor', 'HOOD_ID':..."
...,...,...
153,65e0a11b8e9820685a9dcb1b,"{'AREA_NAME': 'Humbermede', 'HOOD_ID': 22, 'SH..."
154,65e0a11b8e9820685a9dcb1c,"{'AREA_NAME': 'Kingsview Village-The Westway',..."
155,65e0a11b8e9820685a9dcb1d,"{'AREA_NAME': 'Humber Summit', 'HOOD_ID': 21, ..."
156,65e0a11b8e9820685a9dcb1e,"{'AREA_NAME': 'Pelmo Park-Humberlea', 'HOOD_ID..."


In [23]:
#seperating the info in properties column.(REFERENCE:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)
normalised_df=pd.json_normalize(shooting_df['properties'])
shooting_df=pd.concat([shooting_df['_id'], normalised_df],axis=1)
shooting_df

,_id,AREA_NAME,HOOD_ID,SHOOTING_2014,SHOOTING_2015,SHOOTING_2016,SHOOTING_2017,SHOOTING_2018,SHOOTING_2019,SHOOTING_2020,SHOOTING_2021,SHOOTING_2022,SHOOTING_2023
0,65e0a11b8e9820685a9dca82,North Toronto,173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,0.0,2.0,1.0,5.0,3.0,2.0,5.0,3.0,3.0,2.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,1.0,3.0,0.0,3.0,4.0,4.0,2.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,2.0,8.0,3.0,6.0,5.0,4.0,6.0,4.0,2.0,2.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,3.0,7.0,16.0,10.0,3.0,3.0,2.0,7.0,1.0,2.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,1.0,6.0,16.0,9.0,5.0,9.0,5.0,4.0,7.0,6.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,0.0,2.0,5.0,1.0,3.0,3.0,4.0,2.0,3.0,3.0


In [24]:
#getting an overview of the shooting data 
shooting_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   _id            158 non-null    object 
 1   AREA_NAME      158 non-null    object 
 2   HOOD_ID        158 non-null    int64  
 3   SHOOTING_2014  156 non-null    float64
 4   SHOOTING_2015  156 non-null    float64
 5   SHOOTING_2016  156 non-null    float64
 6   SHOOTING_2017  156 non-null    float64
 7   SHOOTING_2018  156 non-null    float64
 8   SHOOTING_2019  156 non-null    float64
 9   SHOOTING_2020  156 non-null    float64
 10  SHOOTING_2021  156 non-null    float64
 11  SHOOTING_2022  156 non-null    float64
 12  SHOOTING_2023  113 non-null    float64
dtypes: float64(10), int64(1), object(2)
memory usage: 16.2+ KB


In [25]:
#getting the total value for shooting
shooting_df['Total_SHOOTING']=shooting_df[['SHOOTING_2014','SHOOTING_2015','SHOOTING_2016','SHOOTING_2017','SHOOTING_2018',
                 'SHOOTING_2019','SHOOTING_2020','SHOOTING_2021','SHOOTING_2022','SHOOTING_2023']].sum(axis=1)
shooting_df

,_id,AREA_NAME,HOOD_ID,SHOOTING_2014,SHOOTING_2015,SHOOTING_2016,SHOOTING_2017,SHOOTING_2018,SHOOTING_2019,SHOOTING_2020,SHOOTING_2021,SHOOTING_2022,SHOOTING_2023,Total_SHOOTING
0,65e0a11b8e9820685a9dca82,North Toronto,173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,2.0
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,6.0
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,5.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,0.0,2.0,1.0,5.0,3.0,2.0,5.0,3.0,3.0,2.0,26.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,1.0,3.0,0.0,3.0,4.0,4.0,2.0,0.0,1.0,2.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,2.0,8.0,3.0,6.0,5.0,4.0,6.0,4.0,2.0,2.0,42.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,3.0,7.0,16.0,10.0,3.0,3.0,2.0,7.0,1.0,2.0,54.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,1.0,6.0,16.0,9.0,5.0,9.0,5.0,4.0,7.0,6.0,68.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,0.0,2.0,5.0,1.0,3.0,3.0,4.0,2.0,3.0,3.0,26.0


In [26]:
#filling null values with 0
shooting_df_clean=shooting_df.fillna(value=0)
shooting_df_clean

,_id,AREA_NAME,HOOD_ID,SHOOTING_2014,SHOOTING_2015,SHOOTING_2016,SHOOTING_2017,SHOOTING_2018,SHOOTING_2019,SHOOTING_2020,SHOOTING_2021,SHOOTING_2022,SHOOTING_2023,Total_SHOOTING
0,65e0a11b8e9820685a9dca82,North Toronto,173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,6.0
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,5.0
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,0.0,2.0,1.0,5.0,3.0,2.0,5.0,3.0,3.0,2.0,26.0
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,1.0,3.0,0.0,3.0,4.0,4.0,2.0,0.0,1.0,2.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,2.0,8.0,3.0,6.0,5.0,4.0,6.0,4.0,2.0,2.0,42.0
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,3.0,7.0,16.0,10.0,3.0,3.0,2.0,7.0,1.0,2.0,54.0
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,1.0,6.0,16.0,9.0,5.0,9.0,5.0,4.0,7.0,6.0,68.0
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,0.0,2.0,5.0,1.0,3.0,3.0,4.0,2.0,3.0,3.0,26.0


In [27]:
#selecting fields for breakenter data 
query={}
fields={"properties.AREA_NAME":1 , "properties.HOOD_ID":1,"properties.BREAKENTER_2014":1,"properties.BREAKENTER_2015":1,
       "properties.BREAKENTER_2016":1,"properties.BREAKENTER_2017":1,"properties.BREAKENTER_2018":1,"properties.BREAKENTER_2019":1,
       "properties.BREAKENTER_2020":1,"properties.BREAKENTER_2021":1,"properties.BREAKENTER_2022":1,"properties.BREAKENTER_2023":1}
break_enter_data=(list(toronto_crime.find(query,fields)))
break_enter_data

[{'_id': ObjectId('65e0a11b8e9820685a9dca82'),
  'properties': {'AREA_NAME': 'North Toronto',
   'HOOD_ID': 173,
   'BREAKENTER_2014': 25,
   'BREAKENTER_2015': 44,
   'BREAKENTER_2016': 15,
   'BREAKENTER_2017': 24,
   'BREAKENTER_2018': 42,
   'BREAKENTER_2019': 36,
   'BREAKENTER_2020': 88,
   'BREAKENTER_2021': 49,
   'BREAKENTER_2022': 46,
   'BREAKENTER_2023': 31}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca83'),
  'properties': {'AREA_NAME': 'South Eglinton-Davisville',
   'HOOD_ID': 174,
   'BREAKENTER_2014': 27,
   'BREAKENTER_2015': 32,
   'BREAKENTER_2016': 25,
   'BREAKENTER_2017': 37,
   'BREAKENTER_2018': 48,
   'BREAKENTER_2019': 83,
   'BREAKENTER_2020': 70,
   'BREAKENTER_2021': 44,
   'BREAKENTER_2022': 26,
   'BREAKENTER_2023': 43}},
 {'_id': ObjectId('65e0a11b8e9820685a9dca84'),
  'properties': {'AREA_NAME': 'Dovercourt Village',
   'HOOD_ID': 172,
   'BREAKENTER_2014': 38,
   'BREAKENTER_2015': 21,
   'BREAKENTER_2016': 37,
   'BREAKENTER_2017': 39,
   'BREAKENTER_2

In [28]:
#converting the break enter data into a pandas dataframe
break_enter_df=pd.DataFrame(break_enter_data)
break_enter_df

,_id,properties
0,65e0a11b8e9820685a9dca82,"{'AREA_NAME': 'North Toronto', 'HOOD_ID': 173,..."
1,65e0a11b8e9820685a9dca83,"{'AREA_NAME': 'South Eglinton-Davisville', 'HO..."
2,65e0a11b8e9820685a9dca84,"{'AREA_NAME': 'Dovercourt Village', 'HOOD_ID':..."
3,65e0a11b8e9820685a9dca85,"{'AREA_NAME': 'Junction-Wallace Emerson', 'HOO..."
4,65e0a11b8e9820685a9dca86,"{'AREA_NAME': 'Yonge-Bay Corridor', 'HOOD_ID':..."
...,...,...
153,65e0a11b8e9820685a9dcb1b,"{'AREA_NAME': 'Humbermede', 'HOOD_ID': 22, 'BR..."
154,65e0a11b8e9820685a9dcb1c,"{'AREA_NAME': 'Kingsview Village-The Westway',..."
155,65e0a11b8e9820685a9dcb1d,"{'AREA_NAME': 'Humber Summit', 'HOOD_ID': 21, ..."
156,65e0a11b8e9820685a9dcb1e,"{'AREA_NAME': 'Pelmo Park-Humberlea', 'HOOD_ID..."


In [29]:
#seperating the info in properties column.(REFERENCE:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)
normalised_df=pd.json_normalize(break_enter_df['properties'])
break_enter_df=pd.concat([break_enter_df['_id'], normalised_df],axis=1)
break_enter_df

,_id,AREA_NAME,HOOD_ID,BREAKENTER_2014,BREAKENTER_2015,BREAKENTER_2016,BREAKENTER_2017,BREAKENTER_2018,BREAKENTER_2019,BREAKENTER_2020,BREAKENTER_2021,BREAKENTER_2022,BREAKENTER_2023
0,65e0a11b8e9820685a9dca82,North Toronto,173,25,44,15,24,42,36,88,49,46,31
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,27,32,25,37,48,83,70,44,26,43
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,38,21,37,39,50,83,47,19,33,46
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,37,46,41,51,91,103,110,76,56,82
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,69,115,61,125,100,119,149,112,123,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,35,35,24,21,44,62,34,30,35,29
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,38,33,34,39,39,27,14,17,15,40
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,54,44,41,40,56,59,25,41,55,90
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,30,15,24,31,24,23,20,12,32,23


In [30]:
#getting an overview of the shooting data 
break_enter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              158 non-null    object
 1   AREA_NAME        158 non-null    object
 2   HOOD_ID          158 non-null    int64 
 3   BREAKENTER_2014  158 non-null    int64 
 4   BREAKENTER_2015  158 non-null    int64 
 5   BREAKENTER_2016  158 non-null    int64 
 6   BREAKENTER_2017  158 non-null    int64 
 7   BREAKENTER_2018  158 non-null    int64 
 8   BREAKENTER_2019  158 non-null    int64 
 9   BREAKENTER_2020  158 non-null    int64 
 10  BREAKENTER_2021  158 non-null    int64 
 11  BREAKENTER_2022  158 non-null    int64 
 12  BREAKENTER_2023  158 non-null    int64 
dtypes: int64(11), object(2)
memory usage: 16.2+ KB


In [31]:
#getting the total value for breakenter
break_enter_df['Total_BREAKENTER']=break_enter_df[['BREAKENTER_2014','BREAKENTER_2015','BREAKENTER_2016','BREAKENTER_2017','BREAKENTER_2018',
                 'BREAKENTER_2019','BREAKENTER_2020','BREAKENTER_2021','BREAKENTER_2022','BREAKENTER_2023']].sum(axis=1)
break_enter_df

,_id,AREA_NAME,HOOD_ID,BREAKENTER_2014,BREAKENTER_2015,BREAKENTER_2016,BREAKENTER_2017,BREAKENTER_2018,BREAKENTER_2019,BREAKENTER_2020,BREAKENTER_2021,BREAKENTER_2022,BREAKENTER_2023,Total_BREAKENTER
0,65e0a11b8e9820685a9dca82,North Toronto,173,25,44,15,24,42,36,88,49,46,31,400
1,65e0a11b8e9820685a9dca83,South Eglinton-Davisville,174,27,32,25,37,48,83,70,44,26,43,435
2,65e0a11b8e9820685a9dca84,Dovercourt Village,172,38,21,37,39,50,83,47,19,33,46,413
3,65e0a11b8e9820685a9dca85,Junction-Wallace Emerson,171,37,46,41,51,91,103,110,76,56,82,693
4,65e0a11b8e9820685a9dca86,Yonge-Bay Corridor,170,69,115,61,125,100,119,149,112,123,108,1081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,65e0a11b8e9820685a9dcb1b,Humbermede,22,35,35,24,21,44,62,34,30,35,29,349
154,65e0a11b8e9820685a9dcb1c,Kingsview Village-The Westway,6,38,33,34,39,39,27,14,17,15,40,296
155,65e0a11b8e9820685a9dcb1d,Humber Summit,21,54,44,41,40,56,59,25,41,55,90,505
156,65e0a11b8e9820685a9dcb1e,Pelmo Park-Humberlea,23,30,15,24,31,24,23,20,12,32,23,234


EXPORTING THE DATA FRAMES 

In [32]:
auto_theft_df_clean.to_csv("Resources/autotheft.csv", encoding='utf8', index=False)

In [33]:
bike_theft_df_clean.to_csv("Resources/biketheft.csv", encoding='utf8', index=False)

In [34]:
shooting_df_clean.to_csv("Resources/shooting.csv", encoding='utf8', index=False)

In [35]:
break_enter_df.to_csv("Resources/breakenter.csv", encoding='utf8', index=False)